In [38]:
import numpy as np
import os
import glob
import math
import numpy as np
import cv2
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D
from math import log10
import SRCNN

In [50]:
%run SRCNN.ipynb
# --- SRCNN Modeli ve Eğitim Sınıfı ---
class SRCNN_Train:
    def __init__(self, train_path, val_path, nImgs, nEpochs, batch_size, print_every, model_path, output_path):
        self.train_path = train_path
        self.val_path = val_path
        self.nImgs = nImgs
        self.nEpochs = nEpochs
        self.batch_size = batch_size
        self.print_every = print_every
        self.model_path = model_path
        self.output_path = output_path
        self.nIters = int(self.nImgs * self.nEpochs / self.batch_size)

        self.data_files = glob.glob(os.path.join(self.train_path, 'Data', '*.png'))[:nImgs]
        self.label_files = glob.glob(os.path.join(self.train_path, 'Labels', '*.png'))[:nImgs]

        print(f"Number of data files: {len(self.data_files)}")
        print(f"Number of label files: {len(self.label_files)}")

        self.mean, self.stddev = self.calculate_mean_stddev()

        self.model = SRCNN()

        # build işlemi için dummy input kullan
        dummy_input = tf.zeros((1, 256, 256, 3))
        _ = self.model(dummy_input)
        self.model.summary()

        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='mean_squared_error')

    def calculate_mean_stddev(self):
        all_images = []
        for file in self.data_files:
            try:
                with open(file, 'rb') as f:
                    file_bytes = np.asarray(bytearray(f.read()), dtype=np.uint8)
                    img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
                if img is not None:
                    img = cv2.resize(img, (256, 256))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Ekleyin
                    all_images.append(img.astype(np.float32))
            except Exception as e:
                print(f"Hata oluştu: {file} - {e}")

        if len(all_images) == 0:
            raise ValueError("Hiçbir görüntü okunamadı.")

        all_images = np.array(all_images)
        mean = np.mean(all_images, axis=(0, 1, 2))
        stddev = np.std(all_images, axis=(0, 1, 2))
        print(f"Calculated mean: {mean}")
        print(f"Calculated stddev: {stddev}")
        return mean, stddev

    def read_batch(self):
        idx = np.random.choice(len(self.data_files), self.batch_size, replace=False)
        imgs, labels = [], []

        for i in idx:
            try:
                with open(self.data_files[i], 'rb') as f:
                    img_bytes = np.asarray(bytearray(f.read()), dtype=np.uint8)
                    img = cv2.imdecode(img_bytes, cv2.IMREAD_COLOR)

                with open(self.label_files[i], 'rb') as f:
                    label_bytes = np.asarray(bytearray(f.read()), dtype=np.uint8)
                    label = cv2.imdecode(label_bytes, cv2.IMREAD_COLOR)

                if img is None or label is None:
                    continue

                img = cv2.resize(img, (256, 256))
                label = cv2.resize(label, (256, 256))

                imgs.append(img.astype(np.float32))
                labels.append(label.astype(np.float32))
            except Exception as e:
                print(f"Görüntü okunamadı: {self.data_files[i]} veya {self.label_files[i]} - {e}")

        imgs = (np.array(imgs) - self.mean) / self.stddev
        labels = (np.array(labels) - self.mean) / self.stddev
        return imgs, labels

    def denormalize(self, img):
        return np.clip((img * self.stddev + self.mean), 0, 255).astype(np.uint8)

    def train(self):
        os.makedirs(self.output_path, exist_ok=True)
        print(">>> Eğitim başlatıldı")

        for epoch in range(self.nEpochs):
            print(f"\nEpoch {epoch+1}/{self.nEpochs}")
            for i in range(self.nIters):
                imgs_inp, imgs_lab = self.read_batch()
                imgs_inp_resized = tf.image.resize(imgs_inp, (256, 256))

                self.model.train_on_batch(imgs_inp_resized, imgs_lab)

                if (i + 1) % self.print_every == 0:
                    loss = self.model.evaluate(imgs_inp_resized, imgs_lab, verbose=0)
                    print(f"[{i+1}/{self.nIters}] Loss: {loss:.4f}")

        print("\n>>> Eğitim tamamlandı.")
        self.model.save(self.model_path.replace(".h5", ".keras"))
        print(f"Model '{self.model_path}' olarak kaydedildi.")
        self.predict()  # Tahmin işlemini başlatıyoruz.

    def predict(self):
        val_files = glob.glob(os.path.join(self.val_path, "*.png"))
        print(f"{len(val_files)} adet validasyon görseli bulundu.")
        
        mean, stddev = self.mean, self.stddev
        self.model.load_weights(self.model_path.replace(".h5", ".keras"))
        print("Model ağırlıkları başarıyla yüklendi.")
        
        # label_dir dizininin son kısmındaki "Data"yı "Labels" ile değiştir
        label_dir = os.path.join(os.path.dirname(self.val_path), 'Labels')
    
        psnr_total = 0.0
        count = 0
        
        for file in val_files:
            try:
                base_name = os.path.basename(file)  # Örn: img_12.png
    
                # Görseli PIL ile oku
                img = Image.open(file)
                if img is None:
                    print(f"Geçersiz görüntü: {file}")
                    continue
    
                # Eğer görüntü RGBA (4 kanal) ise, sadece RGB'yi al
                if img.mode == 'RGBA':
                    img = img.convert('RGB')
    
                img = img.resize((256, 256))  # Görseli yeniden boyutlandır
                img_rgb = np.array(img)  # RGB formatında numpy dizisine çevir
                img_input = ((img_rgb.astype(np.float32)) - mean) / stddev
                img_input = np.expand_dims(img_input, axis=0)
    
                # Tahmin yap
                pred = self.model.predict(img_input)[0]
                pred_img = self.denormalize(pred)
    
                # Tahmini kaydet
                save_path = os.path.join(self.output_path, f"pred_{base_name}")
                pred_pil = Image.fromarray(pred_img.astype(np.uint8))
                pred_pil.save(save_path)
    
                # --- PSNR hesapla ---
                label_path = os.path.join(label_dir, base_name)
                if not os.path.exists(label_path):
                    print(f"🚫 Eşleşen label yok: {label_path}")
                    continue
    
                # Label görselini PIL ile oku
                label_img = Image.open(label_path)
                if label_img is None:
                    print(f"❌ Label okunamadı: {label_path}")
                    continue
    
                # Eğer label görseli RGBA ise, sadece RGB'yi al
                if label_img.mode == 'RGBA':
                    label_img = label_img.convert('RGB')
    
                # Label görselini yeniden boyutlandır
                label_img = label_img.resize((256, 256))
                label_img = np.array(label_img)
    
                # Her iki görüntüyü yeniden boyutlandır ve PSNR hesapla
                pred_img = np.array(pred_img)
                mse = np.mean((pred_img.astype(np.float32) - label_img.astype(np.float32)) ** 2)
                psnr = 100 if mse == 0 else 10 * log10((255 ** 2) / mse)
    
                psnr_total += psnr
                count += 1
    
                print(f"✔ {base_name} - PSNR: {psnr:.2f} dB - Kaydedildi: {save_path}")
    
            except Exception as e:
                print(f"Hata: {file} - {e}")
    
        if count > 0:
            avg_psnr = psnr_total / count
            print(f"\n📊 Ortalama PSNR: {avg_psnr:.2f} dB ({count} görsel üzerinden)")
        else:
            print("🚫 Hiçbir görsel işlenemedi.")

if __name__ == "__main__":
    train_model = SRCNN_Train(
        train_path=r"C:\Users\melis\OneDrive\Masaüstü\image_processing_project\Dataset\Train",
        val_path=r"C:\Users\melis\OneDrive\Masaüstü\image_processing_project\Dataset\Val\Data",
        nImgs=684,
        nEpochs=10,
        batch_size=16,
        print_every=10,
        model_path=r"C:\Users\melis\OneDrive\Masaüstü\image_processing_project\Super-Resolution-master\trained_SRCNN_model.h5",
        output_path=r"C:\Users\melis\OneDrive\Masaüstü\image_processing_project\Super-Resolution-master\Predictions"
    )
    train_model.train()  

Number of data files: 684
Number of label files: 684
Calculated mean: [95.81287 95.81287 95.81287]
Calculated stddev: [69.78387  67.10136  69.271675]


Model: "srcnn_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_87 (Conv2D)                   │ (1, 256, 256, 64)           │          15,616 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_88 (Conv2D)                   │ (1, 256, 256, 32)           │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_89 (Conv2D)                   │ (1, 256, 256, 3)            │           2,403 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,099 (78.51 KB)

 Trainable params: 20,099 (78.51 KB)

 Non-trainable params: 0 (0.00 B)

>>> Eğitim başlatıldı

Epoch 1/10
[10/427] Loss: 0.6892
[20/427] Loss: 0.2580
[30/427] Loss: 0.1484
[40/427] Loss: 0.1297
[50/427] Loss: 0.1434
[60/427] Loss: 0.1316
[70/427] Loss: 0.1044
[80/427] Loss: 0.1023
[90/427] Loss: 0.1349
[100/427] Loss: 0.1000
[110/427] Loss: 0.0736
[120/427] Loss: 0.0904
[130/427] Loss: 0.0984
[140/427] Loss: 0.0787
[150/427] Loss: 0.0592
[160/427] Loss: 0.0715
[170/427] Loss: 0.0611
[180/427] Loss: 0.0640
[190/427] Loss: 0.0566
[200/427] Loss: 0.0530
[210/427] Loss: 0.0849
[220/427] Loss: 0.0493
[230/427] Loss: 0.0541
[240/427] Loss: 0.0473
[250/427] Loss: 0.0632
[260/427] Loss: 0.0535
[270/427] Loss: 0.0458
[280/427] Loss: 0.0500
[290/427] Loss: 0.0484
[300/427] Loss: 0.0493
[310/427] Loss: 0.0435
[320/427] Loss: 0.0438
[330/427] Loss: 0.0401
[340/427] Loss: 0.0419
[350/427] Loss: 0.0384
[360/427] Loss: 0.0443
[370/427] Loss: 0.0454
[380/427] Loss: 0.0351
[390/427] Loss: 0.0317
[400/427] Loss: 0.0359
[410/427] Loss: 0.0439
[420/427] Loss: 0.0471

Epoch 2/